# 7) FUNCIONES DESARROLLADAS PARA EL PROYECTO

> En este notebook se encuentran todas las funciones utilizadas a lo largo de los distintos bloques del proyecto.

- author: Iván Fernández Aguirre
- toc: true
- image: images/N7.png
- sticky_rank: 7

## FUNCIONES GENERALES

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import re  

In [3]:
def rename_columns(DF,List):
    '''
    Cambia nombre de columnas de un Dataframe
    
    Arguments:
        DF: DataFrame
        List: Lista con nombres de columnas
    '''
    d=dict(zip(list(DF.columns),List))
    DF.rename(columns=d, inplace=True)
    return

In [4]:
def check_NaNs(D):
    '''
    Verifica los missing values en un Dataframe
    
    Arguments:
        D: Dataframe
    '''
    l=len(D)
    t=0
    for c in D.columns:
        n=D[c].isna().sum()  
        t=t+n
        if(n>0):
            print(c,'  Nulls: {} / {} - {}%'.format(n,l,np.round(100*n/l,2)))
    if(t>0):
        print('\ntotal missing values: {}'.format(t))
    else:
        print('No hay missing values en el dataset :)')
    return

In [5]:
def Print_features(DF):
    '''
    Imprime por pantalla los Features del dataset FMP, hasta el nootebook 4
    
    Arguments:
        DF: Dataframe
    '''
    print('Today Features\n')
    for c in DF.columns:
        if ( not(bool(re.search(r'\d', c)))):
            print(c)

    print('\n\nHistoric Features\n')
    for c in DF.columns:
        if ( ('1' in c) and (not('0' in c)) and ('home' in c) and (not('away' in c)) ):
            print('home/away'+c[4:-1]+'i')
    return

In [6]:
def Print_features_data_set_modelos(DF):
    '''
    Imprime por pantalla los Features del dataset FMP, para el nootebook 5
    
    Arguments:
        DF: Dataframe
    '''
    print('Today Features\n')
    for c in DF.columns:
        if ( not(bool(re.search(r'\d', c)))):
            print(c)

    print('\n\nHistoric Features\n')
    for c in DF.columns:
        if ( ('1' in c) and (not('0' in c)) and ('Equipo_A' in c) and (not('Equipo_B' in c)) ):
            print('Equipo_(A/B)'+c[8:-1]+'i')
    return

## VISULAIZACIÓN Y EXPLORACIÓN DEL *DATASET*

### Distribuciones de probabilidad e histogramas

In [7]:
from scipy.stats import skellam, norm

In [8]:
def Poisson_distribution(x,l):
    '''
    Forma analítica de la distribución de Poisson 
    
    Arguments:
        l: Lambda
    
    Returns:
        array transformado
    '''
    fact=np.vectorize(np.math.factorial, otypes='O')
    return (l**x)*np.exp(-l)/fact(x)

In [9]:
def Log_normal(x,m,s):
    '''
    Forma analítica de la distribución de Log-normal
    
    Arguments:
        m: media
        s: desviación estandar
    
    Returns:
        array transformado
    '''
    return (1.0/(x*s*np.math.sqrt(2*np.pi)))*np.exp(-((np.log(x)-m)**2)/(2*(s**2)))

In [61]:
def Plot_distributions(DS,columns,rows,Features,Type,n_bin,Dist,x_label,y_label,alig):
    '''
    Grafica histogramas de features de un DataFrame. Calcula: media, std, std error, skewness, entre otros.
    Si se propone una distribución entre las disponibles (Log-norm, Poisson, Normal, Skellam), 
    la grafica tomando los valores del histograma.
    
    Arguments:
        DS: DataFrame
        columns: numero de columnas de la figura
        rows: numerod de filas de la figura
        Features: Lista de Features a analizar
        Type: tipo de dato 'historic' or 'match'
        n_bin: Lista con numero de bines del histograma
        Dist: Lista con Distribución a graficar.
        x_label: Lista con nombre del x label de cada subplot
        y_label: Lista con nombre del y label de cada subplot
        alig: alineacion de las barras del histograma 'mid' 'left' or 'right'

    '''
    fig, a = plt.subplots(nrows=rows, ncols=columns)
    if(columns*rows==1):
        a=[a]
    for ax,f,t,b,d,xl,yl,al in zip(a,Features,Type,n_bin,Dist,x_label,y_label,alig):
        if(t=='historic'):
            C=[t+'_'+f+'_'+str(i) for t in ['home','away'] for i in range(1,11)]
        elif(t=='match'):
            C=[f]
        X=DS[C].to_numpy()
        X=np.reshape(X, (np.size(X),1))
        m=np.nanmean(X)
        std=np.nanstd(X)
        me=np.nanmedian(X)
        sk=3*(m-me)/std
        mx=np.nanmax(X)
        mn=np.nanmin(X)
        n=np.sum(~np.isnan(X))
        H =ax.hist(X, bins=b,density=True,align=al, alpha=0.9)
        if(Dist=='Unknown'):
            textstr = '\n'.join((
                r'$\mu=%.3f$' % (m, ),
                r'max$=%.2f$' % (mx, ),
                r'min$=%.2f$' % (mn, ),
                r'$\mathrm{median}=%.2f$' % (me, ),
                r'$\sigma=%.2f$' % (std, ),
                r'pearson skewness$=%.2f$' % (sk, ),
                r'n$=%.0f$' % (n, ),
                r'std error$=%.3f$' % (std/np.math.sqrt(n), )))
        else:
            textstr = '\n'.join((
                r'$\mu=%.3f$' % (m, ),
                r'max$=%.2f$' % (mx, ),
                r'min$=%.2f$' % (mn, ),
                r'$\mathrm{median}=%.2f$' % (me, ),
                r'$\sigma=%.2f$' % (std, ),
                r'pearson skewness$=%.2f$' % (sk, ),
                r'n$=%.0f$' % (n, ),
                r'std error$=%.3f$' % (std/np.math.sqrt(n), ),
                r'pdf utilizada$:$ '+d))
        
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        ax.text(0.65, 0.95, textstr, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)
        if (d!='Unknown'):
            if(d=='Poisson'):
                B=np.arange(mn,mx+1)
                ax.plot(B,Poisson_distribution(B,m),color='orange',lw=2)
            elif(d=='Log-norm'):
                B=np.linspace(mn,mx,1000)
                m_l=np.math.log(m**2/(np.math.sqrt(m**2+std*2)))
                std_l=np.math.sqrt(np.math.log(1+((std**2)/(m**2))))
                ax.plot(B,Log_normal(B,m_l,std_l),color='orange',lw=2)
            elif(Dist=='Skellam'):
                mu1=(std*2+m)/2
                mu2=(std*2-m)/2
                B=np.arange(mn,mx+1)
                ax.plot(B,skellam.pmf(B,mu1,mu2),color='orange',lw=2)
            elif(Dist=='Normal'):
                B=np.linspace(mn,mx,1000)
                ax.plot(B,norm.pdf(B, m, std),color='orange',lw=2)
        ax.set_ylabel(yl,fontsize=20)
        ax.set_xlabel(xl,fontsize=20)
        ax.tick_params(axis='both', which='major', labelsize=15)
    fig.set_size_inches(20,5)
    plt.tight_layout()
    plt.show()
    return

In [11]:
def Plot_one_distribution(DS,Features,Type,n_bin,Dist,x_label,y_label,al,text_x_pos=0.65,xlim=(None,None)):
    '''
    Grafica el histograma para una featuree de un DataFrame. Calcula: media, std, std error, skewness, entre otros.
    Si se propone una distribución entre las disponibles (Log-norm, Poisson, Normal, Skellam), 
    la grafica tomando los valores del histograma.
    
    Arguments:
        DS: DataFrame
        Feature: Feature a analizar
        Type: tipo de dato 'historic' or 'match'
        n_bin: Lista con numero de bines del histograma
        Dist: Lista con Distribución a graficar.
        x_label: Lista con nombre del x label de cada subplot
        y_label: Lista con nombre del y label de cada subplot
        al: alineacion de las barras del histograma 'mid' 'left' or 'right'
        text_box_pos: posicion [x,y] del cuadro de texto
        xlim: rango en x

    '''
    fig, ax = plt.subplots()
    C=[]
    for f,t in zip(Features,Type):
        if(t=='historic'):
            C=C+[t+'_'+f+'_'+str(i) for t in ['home','away'] for i in range(1,11)]
        elif(t=='match'):
            C=C+[f]
    X=DS[C].to_numpy()
    X=np.reshape(X, (np.size(X),1))
    m=np.nanmean(X)
    std=np.nanstd(X)
    me=np.nanmedian(X)
    sk=3*(m-me)/std
    mx=np.nanmax(X)
    mn=np.nanmin(X)
    n=np.sum(~np.isnan(X))
    H =ax.hist(X, bins=n_bin,density=True,align=al, alpha=0.9)
    if (Dist=='Unknown'):
        textstr = '\n'.join((
            r'$\mu=%.3f$' % (m, ),
            r'max$=%.2f$' % (mx, ),
            r'min$=%.2f$' % (mn, ),
            r'$\mathrm{median}=%.2f$' % (me, ),
            r'$\sigma=%.2f$' % (std, ),
            r'pearson skewness$=%.2f$' % (sk, ),
            r'n$=%.0f$' % (n, ),
            r'std error$=%.3f$' % (std/np.math.sqrt(n), )))
    else:
        textstr = '\n'.join((
            r'$\mu=%.3f$' % (m, ),
            r'max$=%.2f$' % (mx, ),
            r'min$=%.2f$' % (mn, ),
            r'$\mathrm{median}=%.2f$' % (me, ),
            r'$\sigma=%.2f$' % (std, ),
            r'pearson skewness$=%.2f$' % (sk, ),
            r'n$=%.0f$' % (n, ),
            r'std error$=%.3f$' % (std/np.math.sqrt(n), ),
            r'pdf utilizada$:$ '+Dist))
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text(text_x_pos, 0.95, textstr, transform=ax.transAxes, fontsize=11,verticalalignment='top', bbox=props)
    if (Dist!='Unknown'):
        if(Dist=='Poisson'):
            B=np.arange(mn,mx+1)
            ax.plot(B,Poisson_distribution(B,m),color='orange',lw=2)
        elif(Dist=='Log-norm'):
            B=np.linspace(mn,mx,1000)
            m_l=np.math.log(m**2/(np.math.sqrt(m**2+std*2)))
            std_l=np.math.sqrt(np.math.log(1+((std**2)/(m**2))))
            ax.plot(B,Log_normal(B,m_l,std_l),color='orange',lw=2)
        elif(Dist=='Skellam'):
            mu1=(std*2+m)/2
            mu2=(std*2-m)/2
            B=np.arange(mn,mx+1)
            ax.plot(B,skellam.pmf(B,mu1,mu2),color='orange',lw=2)
        elif(Dist=='Normal'):
            B=np.linspace(mn,mx,1000)
            ax.plot(B,norm.pdf(B, m, std),color='orange',lw=2)
    ax.set_ylabel(y_label,fontsize=16)
    ax.set_xlabel(x_label,fontsize=16)
    plt.xticks(fontsize=13)
    plt.yticks(fontsize=13)
    if(xlim[0]!=None):
        ax.set_xlim(xlim)
    fig.set_size_inches(12,5)
    plt.show()
    return

In [12]:
def Plot_distribution_por_Condicion_team_and_opp(DS,Features_Team,Features_Opponent,Condition,Cond_Labels,n_bin,Dist,x_label,y_label,al,Text_box_pos=[0.4,0.7]):
    '''
    Grafica dos histogramas de una feature en función de una condicion donde sumamos datos historicos de equipos y oponentes.
    Calcula: media, std, std error, skewness, entre otros. Si se propone una distribución entre las disponibles (Log-norm, Poisson, Normal, Skellam), 
    la grafica tomando los valores del histograma.
    
    Arguments:
        DS: DataFrame
        Features_team: Feature a analizar (team)
        Features_Opponent: Feature a analizar (historic opponent)
        Condition: Condicion (variable true or False) que separa los datos
        Condition_Labels: Que representa True y False
        Type: tipo de dato 'historic' or 'match'
        n_bin: Lista con numero de bines del histograma
        Dist: Lista con Distribución a graficar.
        x_label: Lista con nombre del x label de cada subplot
        y_label: Lista con nombre del y label de cada subplot
        al: alineacion de las barras del histograma 'mid' 'left' or 'right'
        text_box_pos: posicion [x,y] del cuadro de texto
        xlim: rango en x

    '''
    
    fig, ax = plt.subplots()
    
    Local=np.array([])
    Visitante=np.array([])
    for t in ['home','away']:
        C=[t+'_'+Condition+'_'+str(i) for i in range(1,11)]
        fil=DS[C].to_numpy()
        fil=np.reshape(fil, (np.size(fil),1)).astype('bool')
        F=[t+'_'+Features_Team+'_'+str(i) for i in range(1,11)]
        G=DS[F].to_numpy()
        G=np.reshape(G, (np.size(G),1))
        
        Local=np.concatenate((Local,G[fil]))
        Visitante=np.concatenate((Visitante,G[~fil]))
    
    for t in ['home','away']:
        C=[t+'_'+Condition+'_'+str(i) for i in range(1,11)]
        fil=DS[C].to_numpy()
        fil=np.reshape(fil, (np.size(fil),1)).astype('bool')
        F=[t+'_'+Features_Opponent+'_'+str(i) for i in range(1,11)]
        G=DS[F].to_numpy()
        G=np.reshape(G, (np.size(G),1))
        
        Local=np.concatenate((Local,G[~fil]))
        Visitante=np.concatenate((Visitante,G[fil]))
        
    
    for X,n,p,b,c in zip([Local,Visitante],Cond_Labels,Text_box_pos,n_bin,['k','r']):
        m=np.nanmean(X)
        std=np.nanstd(X)
        me=np.nanmedian(X)
        sk=3*(m-me)/std
        mx=np.nanmax(X)
        mn=np.nanmin(X)
        nn=np.sum(~np.isnan(X))
        H =ax.hist(X, bins=b,density=True,align=al,alpha=0.5,color=c)
        if (Dist=='Unknown'):
            textstr = '\n'.join((
                n,
                r'$\mu=%.3f$' % (m, ),
                r'max$=%.2f$' % (mx, ),
                r'min$=%.2f$' % (mn, ),
                r'$\mathrm{median}=%.2f$' % (me, ),
                r'$\sigma=%.2f$' % (std, ),
                r'pearson skewness$=%.2f$' % (sk, ),
                r'n$=%.0f$' % (nn, ),
                r'std error$=%.3f$' % (std/np.math.sqrt(nn), )))
        else:
            textstr = '\n'.join((
                n,
                r'$\mu=%.3f$' % (m, ),
                r'max$=%.2f$' % (mx, ),
                r'min$=%.2f$' % (mn, ),
                r'$\mathrm{median}=%.2f$' % (me, ),
                r'$\sigma=%.2f$' % (std, ),
                r'pearson skewness$=%.2f$' % (sk, ),
                r'n$=%.0f$' % (nn, ),
                r'std error$=%.3f$' % (std/np.math.sqrt(nn), ),
                r'pdf utilizada$:$ '+Dist))
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        ax.text(p, 0.95, textstr, transform=ax.transAxes, fontsize=11,verticalalignment='top', bbox=props,color=c)
        if (Dist!='Unknown'):
            if(Dist=='Poisson'):
                B=np.arange(mn,mx+1)
                ax.plot(B,Poisson_distribution(B,m),color=c,lw=2)
            elif(Dist=='Log-norm'):
                B=np.linspace(mn,mx,1000)
                m_l=np.math.log(m**2/(np.math.sqrt(m**2+std*2)))
                std_l=np.math.sqrt(np.math.log(1+((std**2)/(m**2))))
                ax.plot(B,Log_normal(B,m_l,std_l),color=c,lw=2)
            elif(Dist=='Skellam'):
                mu1=(std*2+m)/2
                mu2=(std*2-m)/2
                B=np.arange(mn,mx+1)
                ax.plot(B,skellam.pmf(B,mu1,mu2),color=c,lw=2)
            elif(Dist=='Normal'):
                B=np.linspace(mn,mx,1000)
                ax.plot(B,norm.pdf(B, m, std),color='orange',lw=2)
    ax.set_ylabel(y_label,fontsize=16)
    ax.set_xlabel(x_label,fontsize=16)
    plt.xticks(fontsize=13)
    plt.yticks(fontsize=13)
    fig.set_size_inches(12,5)
    plt.show()
    return

In [13]:
def Plot_distribution_por_Condicion(DS,Feature,Condition,Cond_Labels,n_bin,Dist,x_label,y_label,al,Text_box_pos=[0.4,0.7]):
    '''
    Grafica dos histogramas de una feature en función de una condicion. Calcula: media, std, std error, skewness, entre otros.
    Si se propone una distribución entre las disponibles (Log-norm, Poisson, Normal, Skellam), 
    la grafica tomando los valores del histograma.
    
    Arguments:
        DS: DataFrame
        Features: Feature a analizar
        Condition: Condicion (variable true or False) que separa los datos
        Condition_Labels: Que representa True y False
        Type: tipo de dato 'historic' or 'match'
        n_bin: Lista con numero de bines del histograma
        Dist: Lista con Distribución a graficar.
        x_label: Lista con nombre del x label de cada subplot
        y_label: Lista con nombre del y label de cada subplot
        al: alineacion de las barras del histograma 'mid' 'left' or 'right'
        text_box_pos: posicion [x,y] del cuadro de texto
        xlim: rango en x

    '''
    fig, ax = plt.subplots()
    
    Local=np.array([])
    Visitante=np.array([])
    for t in ['home','away']:
        C=[t+'_'+Condition+'_'+str(i) for i in range(1,11)]
        fil=DS[C].to_numpy()
        fil=np.reshape(fil, (np.size(fil),1)).astype('bool')
        F=[t+'_'+Feature+'_'+str(i) for i in range(1,11)]
        G=DS[F].to_numpy()
        G=np.reshape(G, (np.size(G),1))
        
        Local=np.concatenate((Local,G[fil]))
        Visitante=np.concatenate((Visitante,G[~fil]))

    for X,n,p,b,c in zip([Local,Visitante],Cond_Labels,Text_box_pos,n_bin,['k','r']):
        m=np.nanmean(X)
        std=np.nanstd(X)
        me=np.nanmedian(X)
        sk=3*(m-me)/std
        mx=np.nanmax(X)
        mn=np.nanmin(X)
        nn=np.sum(~np.isnan(X))
        H =ax.hist(X, bins=b,density=True,align=al,alpha=0.5,color=c)
        if (Dist=='Unknown'):
            textstr = '\n'.join((
                n,
                r'$\mu=%.3f$' % (m, ),
                r'max$=%.2f$' % (mx, ),
                r'min$=%.2f$' % (mn, ),
                r'$\mathrm{median}=%.2f$' % (me, ),
                r'$\sigma=%.2f$' % (std, ),
                r'pearson skewness$=%.2f$' % (sk, ),
                r'n$=%.0f$' % (nn, ),
                r'std error$=%.3f$' % (std/np.math.sqrt(nn), )))
        else:
            textstr = '\n'.join((
                n,
                r'$\mu=%.3f$' % (m, ),
                r'max$=%.2f$' % (mx, ),
                r'min$=%.2f$' % (mn, ),
                r'$\mathrm{median}=%.2f$' % (me, ),
                r'$\sigma=%.2f$' % (std, ),
                r'pearson skewness$=%.2f$' % (sk, ),
                r'n$=%.0f$' % (nn, ),
                r'std error$=%.3f$' % (std/np.math.sqrt(nn), ),
                r'pdf utilizada$:$ '+Dist))
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        ax.text(p, 0.95, textstr, transform=ax.transAxes, fontsize=11,verticalalignment='top', bbox=props,color=c)
        if (Dist!='Unknown'):
            if(Dist=='Poisson'):
                B=np.arange(mn,mx+1)
                ax.plot(B,Poisson_distribution(B,m),color=c,lw=2)
            elif(Dist=='Log-norm'):
                B=np.linspace(mn,mx,1000)
                m_l=np.math.log(m**2/(np.math.sqrt(m**2+std*2)))
                std_l=np.math.sqrt(np.math.log(1+((std**2)/(m**2))))
                ax.plot(B,Log_normal(B,m_l,std_l),color=c,lw=2)
            elif(Dist=='Skellam'):
                mu1=(std*2+m)/2
                mu2=(std*2-m)/2
                B=np.arange(mn,mx+1)
                ax.plot(B,skellam.pmf(B,mu1,mu2),color=c,lw=2)
            elif(Dist=='Normal'):
                B=np.linspace(mn,mx,1000)
                ax.plot(B,norm.pdf(B, m, std),color='orange',lw=2)
    ax.set_ylabel(y_label,fontsize=16)
    ax.set_xlabel(x_label,fontsize=16)
    plt.xticks(fontsize=13)
    plt.yticks(fontsize=13)
    fig.set_size_inches(12,5)
    plt.show()
    return

In [76]:
def Histograma_condicional_numerica(DF,variable,variable_condicional,tipo_de_condición,condicion_numerica,x_label,legend_pos='best'):
    '''
    Grafica histograma de la variable, en funcion de la condicion dada.
    
    Arguments:
        DF: DataFrame
        variable: Feature para hacer el histograma
        variable_condicional: feature que servirá para separar los datos
        tipo_de_condición: 'bool' o '><_value'
        condicion_numerica: Condición sobre ese feature
        X_label: Nombre del feature del histograma

    '''
    fig = plt.figure(figsize=(8,5))
    ax = fig.add_subplot(111)
    ind = np.arange(3)
    width = 0.2 
    if(tipo_de_condición=='><_value'):
        ax.bar(ind-width, DF[variable].value_counts(normalize=True) , width, label='General  N='+str(len(DF)), alpha=0.7, color='red')
        ax.bar(ind, DF[variable][DF[variable_condicional]>condicion_numerica].value_counts(normalize=True).reindex(DF[variable].value_counts(normalize=True).index) , width, label=variable_condicional+' > '+str(condicion_numerica)+'  N='+str(np.sum(DF[variable_condicional]>condicion_numerica)), alpha=0.7, color='navy')
        ax.bar(ind+width, DF[variable][DF[variable_condicional]<-condicion_numerica].value_counts(normalize=True).reindex(DF[variable].value_counts(normalize=True).index), width, label=variable_condicional+' < '+str(-condicion_numerica)+'  N='+str(np.sum(DF[variable_condicional]<-condicion_numerica)), alpha=0.7, color='orange')    
    elif(tipo_de_condición=='bool'):
        ax.bar(ind-width, DF[variable].value_counts(normalize=True) , width, label='General  N='+str(len(DF)), alpha=0.7, color='red')
        ax.bar(ind, DF[variable][DF[variable_condicional]==1].value_counts(normalize=True).reindex(DF[variable].value_counts(normalize=True).index) , width, label=variable_condicional+' = True '+'  N='+str(np.sum(DF[variable_condicional]==1)), alpha=0.7, color='navy')
        ax.bar(ind+width, DF[variable][DF[variable_condicional]==0].value_counts(normalize=True).reindex(DF[variable].value_counts(normalize=True).index), width, label=variable_condicional+' = False '+'  N='+str(np.sum(DF[variable_condicional]==0)), alpha=0.7, color='orange')    
    ax.set_xlabel(x_label,fontsize=17)
    ax.set_ylabel('Distribución de Resultados',fontsize=17)
    ax.tick_params(axis='x',labelsize=15,labelrotation=0)
    ax.tick_params(axis='y',labelsize=14)
    plt.legend(fontsize=12,loc=legend_pos,framealpha=0.9)
    ax.set_xticks(ind)
    ax.set_xticklabels( (DF[variable].value_counts(normalize=True).index) )
    plt.show()

In [15]:
def Histograma_VAR_categorica(DF,variable,x_label):
    '''
    Grafica histograma de la variable categorica
    
    Arguments:
        DF: DataFrame
        variable: Feature para hacer el histograma
        X_label: Nombre del feature del histograma

    '''
    ax=DF[variable].value_counts(normalize=True).plot(kind='bar',alpha=0.7, color='blue',figsize=(8,5))
    ax.set_xlabel(x_label,fontsize=17)
    ax.set_ylabel('Distribución de Resultados',fontsize=17)
    ax.tick_params(axis='x',labelsize=15,labelrotation=0)
    ax.tick_params(axis='y',labelsize=14)
    #plt.legend(fontsize=14)
    plt.show()

In [16]:
def Histograma_historic_condition_n_times(DF,variable,variable_condicional,n,x_label,legend_pos='best'):
    '''
    Grafica histograma de la variable, en funcion de la condicion dada.
    
    Arguments:
        DF: DataFrame
        variable: Feature para hacer el histograma
        variable_condicional: feature que servirá para separar los datos
        n: numero de veces
        X_label: Nombre del feature del histograma

    '''
    
    aux=pd.Series(np.zeros(len(DF)),index=DF.index)
    for i in range(1,11):
        l=variable_condicional+'_{}'.format(i)
        aux=aux+DF[l].astype('int')

    fig = plt.figure(figsize=(8,5))
    ax = fig.add_subplot(111)
    ind = np.arange(3)
    width = 0.3 
    
    ax.bar(ind, DF[variable].value_counts(normalize=True) , width, label='General  N='+str(len(DF)), alpha=0.7, color='red')
    ax.bar(ind+width, DF[variable][  aux==n  ].value_counts(normalize=True).reindex(DF[variable].value_counts(normalize=True).index) , width, label=variable_condicional+' is True '+str(n)+' veces.   N='+str(np.sum(aux==n)), alpha=0.7, color='navy')
    
    ax.set_xlabel(x_label,fontsize=17)
    ax.set_ylabel('Distribución de Resultados',fontsize=17)
    ax.tick_params(axis='x',labelsize=15,labelrotation=0)
    ax.tick_params(axis='y',labelsize=14)
    plt.legend(fontsize=12,loc=legend_pos,framealpha=0.9)
    ax.set_xticks(ind+width/2)
    ax.set_xticklabels( (DF[variable].value_counts(normalize=True).index) )
    plt.show()

In [ ]:
def Histograma_historic_2_condition_n_times(DF,variable,tipo,variable_condicional_1,n1,variable_condicional_2,n2,x_label,legend_pos='best'):
    '''
    Grafica histograma de la variable, en funcion de la condicion dada.
    
    Arguments:
        DF: DataFrame
        variable: Feature para hacer el histograma
        tipo: tipo de condición 'both =', 'both >=', '1 >= 2 <='
        variable_condicional_1: feature que servirá para separar los datos
        n1: numero de veces
        variable_condicional_2: feature que servirá para separar los datos
        n2: numero de veces
        X_label: Nombre del feature del histograma

    '''
    
    aux1=pd.Series(np.zeros(len(DF)),index=DF.index)
    for i in range(1,11):
        l=variable_condicional_1+'_{}'.format(i)
        aux1=aux1+DF[l].astype('int')
    
    aux2=pd.Series(np.zeros(len(DF)),index=DF.index)
    for i in range(1,11):
        l=variable_condicional_2+'_{}'.format(i)
        aux2=aux2+DF[l].astype('int')

    fig = plt.figure(figsize=(8,5))
    ax = fig.add_subplot(111)
    ind = np.arange(3)
    width = 0.3 
    
    ax.bar(ind, DF[variable].value_counts(normalize=True) , width, label='General  N='+str(len(DF)), alpha=0.7, color='red')
    if(tipo=='both ='):
        ax.bar(ind+width, DF[variable][  (aux1==n1) &  (aux2==n2)  ].value_counts(normalize=True).reindex(DF[variable].value_counts(normalize=True).index) , width, label=variable_condicional_1+' is True '+str(n1)+' veces\n'+variable_condicional_2+' is True '+str(n2)+' veces\n'+   'N='+str(np.sum((aux1==n1)&(aux2==n2))), alpha=0.7, color='navy')
    elif(tipo=='both >='):    
        ax.bar(ind+width, DF[variable][  (aux1>=n1) &  (aux2>=n2)  ].value_counts(normalize=True).reindex(DF[variable].value_counts(normalize=True).index) , width, label=variable_condicional_1+' is True '+str(n1)+' veces al menos\n'+variable_condicional_2+' is True '+str(n2)+' veces al menos\n'+   'N='+str(np.sum((aux1>=n1)&(aux2>=n2))), alpha=0.7, color='navy')
    elif(tipo=='1 >= 2 <='):    
        ax.bar(ind+width, DF[variable][  (aux1>=n1) &  (aux2<=n2)  ].value_counts(normalize=True).reindex(DF[variable].value_counts(normalize=True).index) , width, label=variable_condicional_1+' is True '+str(n1)+' veces al menos\n'+variable_condicional_2+' is True '+str(n2)+' veces al menos\n'+   'N='+str(np.sum((aux1>=n1)&(aux2<=n2))), alpha=0.7, color='navy')
    
    
    ax.set_xlabel(x_label,fontsize=17)
    ax.set_ylabel('Distribución de Resultados',fontsize=17)
    ax.tick_params(axis='x',labelsize=15,labelrotation=0)
    ax.tick_params(axis='y',labelsize=14)
    plt.legend(fontsize=12,loc=legend_pos,framealpha=0.9)
    ax.set_xticks(ind+width/2)
    ax.set_xticklabels( (DF[variable].value_counts(normalize=True).index) )
    plt.show()

### Matrices de correlación

In [17]:
import seaborn as sns

In [18]:
def Matrices_de_correlacion_All_Features_a_la_vez(DS):
    '''
    Genera matrices de correlacion para todas las features del dataset.
    Las referentes al partido a modelar, las agrupa en un solo subplot.
    Para las históricas, se genera un subplot para cada una.
    
    Arguments:
        DS: DataFrame
        
    '''
    n_DS= pd.get_dummies(DS, columns=['target'])
    Lista=list(n_DS.columns)[:-3]+['Victoria visitante','Empate','Victoria local']
    rename_columns(n_DS,Lista)
 
    Match_day_features=[]
    for c in n_DS.columns:
        if ( (c!='id') and not(bool(re.search(r'\d', c)))):
            Match_day_features.append(c)
    corrMatrix_matchday =n_DS[Match_day_features].corr()
    
    M={}
    Features=['goal_diff','rating_diff','coach_continuity','team_history_is_play_home','team_history_is_cup']
    for f in Features:
        Col=[]
        for t in ['home','away']:
            for i in range(1,11):
                c=t+'_'+f+'_{}'.format(i)
                Col.append(c)
        Col=Col+['Victoria visitante','Empate','Victoria local']
        M[f]=n_DS[Col].corr()
    
    
    
    fig, axes = plt.subplots(3, 2, figsize=(25,30))
    axes=np.reshape(axes,(6,))
    
    axes[0].set_title(r'$sign(M)\,\sqrt{abs(M)}$'+', siendo M la matriz de correlacion para las variables del partido\n', fontsize=14)
    sns.heatmap(np.sign(corrMatrix_matchday)*np.sqrt(np.abs(corrMatrix_matchday)), annot=True,ax=axes[0], cmap="vlag")
    
    for a,f in zip(axes[1:],Features):
        a.set_title(r'$sign(M)\,\sqrt{abs(M)}$'+', siendo M la matriz de correlacion para las variables "{}"\n'.format(f.replace('_', ' ')), fontsize=14)
        sns.heatmap(np.sign(M[f])*np.sqrt(np.abs(M[f])), annot=True,ax=a , cmap="vlag")
    plt.tight_layout()
    plt.show()
    return

In [19]:
def Matrices_de_correlacion_All_Features(DS,tipo,historic_feature=None):
    '''
    Genera una matriz de correlacion para las features seleccionadas. 
    
    Arguments:
        DS: DataFrame
        tipo: 'Match_day_Features' para las del partido a modelar, o 'Historicas' para una feature historica
        historic_feature: nombre de la feature cuando es de categoria 'Historicas'
        
    '''
    
    n_DS= pd.get_dummies(DS, columns=['target'])
    Lista=list(n_DS.columns)[:-3]+['Victoria visitante','Empate','Victoria local']
    rename_columns(n_DS,Lista)
    
    if(tipo=='Match_day_Features'):
        Col=[]
        for c in n_DS.columns:
            if ( (c!='id') and not(bool(re.search(r'\d', c)))):
                Col.append(c)
    elif(tipo=='Historicas'):
        if(historic_feature==None):
            print('Historic feature needs to be specify as an additional argument')
            return
        f=historic_feature
        Col=[]
        for t in ['home','away']:
            for i in range(1,11):
                c=t+'_'+f+'_{}'.format(i)
                Col.append(c)
        Col=Col+['Victoria visitante','Empate','Victoria local']
        
    M=n_DS[Col].corr()
    
    fig, axes = plt.subplots(1, 1, figsize=(18,8))

    if(tipo=='Match_day_Features'):
        label='del partido'
    elif(tipo=='Historicas'):
        label='historicas de '+historic_feature.replace('_', ' ')
    
    axes.set_title(r'$sign(M)\,\sqrt{abs(M)}$'+', siendo M la matriz de correlacion para las variables "{}"\n'.format(label), fontsize=18)
    sns.heatmap(np.sign(M)*np.sqrt(np.abs(M)), annot=True,ax=axes, cmap='vlag' )
    
    plt.tight_layout()
    plt.show()
    return

### Otros

In [20]:
def Best_rating_teams(DS,N,league=[-1,'None']):
    '''
    Genera una tabla (DataFrame) con los mejores N equipos de una liga, o en su defecto del dataset.
    
    Arguments:
        DS: DataFrame
        N: numero de equipos en la lista
        league: [id,nombre]

    '''
    f='team_history_rating'
    c_h=[t+'_'+f+'_'+str(i) for t in ['home'] for i in range(1,11)]
    C_home=['home_team_name']+c_h
    
    c_a=[t+'_'+f+'_'+str(i) for t in ['away'] for i in range(1,11)]
    C_away=['away_team_name']+c_a
    
    idd=league[0]
    if(idd==-1):
        print('Todas las Ligas del Dataset\n')
        DS_2=DS
    elif(idd in DS.league_id.unique()):
        print('Liga: '+league[1])
        DS_2=DS[DS.league_id==idd]
        
    X_home=DS_2[C_home]
    X_home['Rank']=X_home[c_h].mean(axis=1)
    X_home=X_home.drop(columns=c_h)

    X_away=DS_2[C_away]
    X_away['Rank']=X_away[c_a].mean(axis=1)
    X_away=X_away.drop(columns=c_a)
    
    rename_columns(X_home,['Equipo','Rating'])
    rename_columns(X_away,['Equipo','Rating'])
    
    X = pd.concat([X_home,X_away])

    X=X.groupby('Equipo').mean()
    
    X=X.sort_values('Rating',ascending=False)
    
    display(X.head(N))
    return

In [21]:
def Correlacion_X_vs_Y_his(DS,X_label,Y_label,Plot_labels,alpha=1,text_x_pos=0.7):
    '''
    Grafica el scatter plot de dos features y calcula su correlación.
    
    Arguments:
        DS: DataFrame
        X_label: Feature X
        Y_label: Feature Y
        Plot_labels= nombre de los labels [x,y]
        alpha: nivel de trasparencia de los puntos
        x_pos: posicion x del cuadro de texto donde aparece el valor de correlacion

    '''
    X_c=[]
    for xl in X_label:
        X_c=X_c+[t+'_'+xl+'_'+str(i) for t in ['home','away'] for i in range(1,11)]
    X=DS[X_c].to_numpy()
    X=np.reshape(X, (np.size(X),))
    
    Y_c=[]
    for yl in Y_label:
        Y_c=Y_c+[t+'_'+yl+'_'+str(i) for t in ['home','away'] for i in range(1,11)]
    Y=DS[Y_c].to_numpy()
    Y=np.reshape(Y, (np.size(Y),))
    
    corr=np.ma.corrcoef(np.ma.masked_invalid(X), np.ma.masked_invalid(Y))
    
    fig, ax = plt.subplots()
    ax.set_xlabel(Plot_labels[0],fontsize=17)
    ax.set_ylabel(Plot_labels[1],fontsize=17)
    ax.tick_params(axis='both', which='major', labelsize=17)
    ax.scatter(X,Y,alpha=alpha)
    textstr = r'$Correlation\,\,Value=%.3f$' % (corr[0,1], )
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text(text_x_pos, 0.95, textstr, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props,color='k')
    fig.set_size_inches(20,5)
    plt.tight_layout()
    plt.show()
    return

In [22]:
def Plot_Outcome_para_copa_y_liga(DS):
    '''
    Realiza un grafico de barras con los porcentages de Victoria, Empate y Derrota para partidos de: Copa, Liga y General.
    
    Arguments:
        DS: DataFrame
        
    '''
    Total=DS.target.value_counts()/len(DS.target)
    Copa=DS[DS.is_cup==True].target.value_counts()/len(DS[DS.is_cup==True])
    Liga=DS[DS.is_cup==False].target.value_counts()/len(DS[DS.is_cup==False])
    
    X=np.arange(3)
    fig, ax = plt.subplots()
    ax.set_ylabel('Distribución de Resultados',fontsize=17)
    ax.set_xlabel('Resultado',fontsize=17)
    ax.bar(X-0.11,Total.values,width=0.1,label='General')
    ax.bar(X,Liga.values,width=0.1,label='Liga')
    ax.bar(X+0.11,Copa.values,width=0.1,label='Copa')
    
    xt=[item.get_text() for item in ax.get_xticklabels()]
    xt[1]=Total.index[0]
    xt[3]=Total.index[1]
    xt[5]=Total.index[2] 
    ax.set_xticklabels(xt)
    ax.tick_params(axis='both', which='major', labelsize=17)
    
    plt.legend()
    fig.set_size_inches(10,5)
    plt.show()

    return

In [23]:
def Match_dates_histograms(DS):
    '''
    Genera un histograma con las fechas de los partidos, separando partidos a modelar de los hitoricos del input.
    
    Arguments:
        DS: DataFrame
        
    '''
    fig, ax = plt.subplots()
    
    Feature='team_history_match_date'
    C=[t+'_'+Feature+'_'+str(i) for t in ['home','away'] for i in range(1,11)]
    X=DS[C].to_numpy()
    X=np.reshape(X, (np.size(X),1))
    
    H=pd.DataFrame(X).dropna()
    rename_columns(H,['Date'])
    H.Date=H.Date.astype("datetime64")
    H['Year']= H.Date.dt.year
    H['Month']=H.Date.dt.month
    Values_Historic=H.groupby(['Year','Month']).count()
    Values_Historic=100*Values_Historic/Values_Historic.Date.sum()
    
    X1=np.arange(len(Values_Historic))
    Y1=Values_Historic.values[:,0]
    ax.bar(X1,Y1,color='black',alpha=0.3,label='Partidos Historial')
    
    
    T = DS.match_date.astype("datetime64")
    T=pd.DataFrame(T)
    rename_columns(T,['Date'])
    T['Year']=T.Date.dt.year
    T['Month']=T.Date.dt.month
    Values_Match_day=T.groupby(['Year','Month']).count()
    Values_Match_day=100*Values_Match_day/Values_Match_day.Date.sum()
    
    
    X2=np.arange(len(Values_Match_day))+11
    Y2=Values_Match_day.values[:,0]
    ax.bar(X2,Y2,color='red',alpha=0.3,label='Partidos a Modelar')
    
    
    ax.set_ylabel('Porcentage de Partidos (%)',fontsize=17)
    ax.set_xlabel('Fecha',fontsize=17)
    plt.legend()
    
    
    ax.set_xticks(np.arange(min(X1), max(X2)+1, 1.0))
    
    xt=[item.get_text() for item in ax.get_xticklabels()]
    xt=list(Values_Historic.index)
    xt.append((Values_Match_day.index)[-1])
    ax.set_xticklabels(xt,rotation=90)
    ax.tick_params(axis='both', which='major', labelsize=15)
    fig.set_size_inches(10,5)
    plt.show()
    
    return

In [78]:
def Trainer_effect_on_the_outcome(DS,dias_ventana):
    '''
    Analiza el efecto de cambios de entrenadores en termino de porcentage de victorias, derrotas y empates, 
    y el tiempo transcurrido desde dicho evento. Realiza un grafico y una tabla donde se muestran los resultados.
    
    Arguments:
        DS: DataFrame
        dias_ventana: número de dias a analizar alrrededor del cambio de entrenador
        
    '''
    K=list(np.arange(-dias_ventana,dias_ventana+1))#[-3,-2,-1,0,1,2,3]
    Victorias=[]
    Derrotas=[]
    Empates=[]
    Muestras=[]
    Local=[]
    for k in K:
        C=np.array([])
        D=np.array([])
        V=np.array([])
        L=np.array([])
        for t in ['home','away']:
            for i in range(max(1,1-k),min(10,10-k)):
                c=t+'_coach_continuity_{}'.format(i)
                d=t+'_outcome_V_{}'.format(i+k)
                v=t+'_outcome_D_{}'.format(i+k)
                l=t+'_team_history_is_play_home_{}'.format(i)
                C=np.concatenate((C,DS[c].to_numpy()))
                D=np.concatenate((D,DS[d].to_numpy()))
                V=np.concatenate((V,DS[v].to_numpy()))
                L=np.concatenate((L,DS[l].to_numpy()))
        n=len(C[C==0])
        p_v=np.sum(V[C==0])/n
        p_d=np.sum(D[C==0])/n
        p_e=1-p_d-p_v
        p_l=np.sum(L[C==0])/n
        Victorias.append(p_v)
        Empates.append(p_e)
        Derrotas.append(p_d)
        Muestras.append(n)
        Local.append(p_l)
        if(k==0):
            n=len(C)
            p_v=np.sum(V)/n
            p_d=np.sum(D)/n
            p_e=1-p_d-p_v
            p_l=np.sum(L)/n
            Victorias=[p_v]+Victorias
            Empates=[p_e]+Empates
            Derrotas=[p_d]+Derrotas
            Muestras=[n]+Muestras
            Local=[p_l]+Local
    LABEL=['General']
    for k in K:
        if(k<0):
            LABEL=LABEL+['{} partidos antes de un cambio de entrenador'.format(abs(k))]
        elif(k>0):
            LABEL=LABEL+['{} partidos después de un cambio de entrenador'.format(abs(k))]
        elif(k==0):
            LABEL=LABEL+['En el partido de un cambio de entrenador'.format(abs(k))]
    df={'Condición':LABEL,
        'Muestras': Muestras,
        '% Local': np.round(100*np.array(Local),2),
        '% Victorias':np.round(100*np.array(Victorias),2),
        '% Empates:':np.round(100*np.array(Empates),2),
        '% Derrotas':np.round(100*np.array(Derrotas),2)}
    df=pd.DataFrame(df)
    display(df)
    
    fig, ax = plt.subplots()
    ax.set_xlabel('Partidos hasta cambio de Entrenador',fontsize=16)
    ax.set_ylabel('Distribución de Resultados',fontsize=16)
    ax.plot(K,Victorias[1:],'-o',label='% Victorias')
    ax.plot(K,Empates[1:],'-o',label='% Empates')
    ax.plot(K,Derrotas[1:],'-o',label='% Derrotas')
    ax.axvline(0,ls='--',lw=2,c='darkgrey')
    ax.axhline(Victorias[0],ls='--',lw=1,c='b')
    ax.axhline(Empates[0],ls='--',lw=1,c='orange')
    ax.axhline(Derrotas[0],ls='--',lw=1,c='green')
    textstr = 'Las líneas punteadas indican\nlos valores en el data set completo'
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text(0.01, 0.98, textstr, transform=ax.transAxes, fontsize=13,verticalalignment='top', bbox=props,color='k')
    plt.xticks(fontsize=13)
    plt.yticks(fontsize=13)
    plt.legend(loc=(0.01, 0.3), prop={'size': 14})
    fig.set_size_inches(12,5)
    plt.tight_layout()
    plt.show()
    return

## TRANSFORMACIÓN DEL DATASET PARA LOS MODELOS

### Balance de targets

In [25]:
def Classes_counts(DS):
    '''
    Imprime por pantalla las clases en el Dataframe de target y su ocurrencia/frecuencia.
    
    Arguments:
        DS: DataFrame
        
    '''
    v,q=np.unique(DS.target,return_counts=True)
    n=np.sum(q)
    ddict=dict(zip(v,q))
    print('CLASES\n')
    for k in ddict.keys():
        print(k,': ',ddict[k],' ',round(100*ddict[k]/n,2),'%')
    return

In [26]:
def data_balancing_Victorias_y_Derrotas(DS):
    '''
    Balancea las clases "away" y "home" a través de intercambiar visitantes y locales. Ahora pasan a llamarse Equipo 1 y
    Equipo 2, y la información de localía queda condensada en la variable "EqA_Local". También realiza un mezclado de las
    muestras.
    
    Devuelve el Dataframe resultante.
    
    Arguments:
        DS: DataFrame
        
    Returns:
        DS_2: Dataframe resultante
    '''
    #This function swaps 0.5 of the home and away, and oversamples draws 
    ###################################################################
    
    DS_2=DS.copy(deep=True)
    DS_2['EqA_Local']=np.ones(len(DS_2)).astype('int')
    DS_2['swap']=np.random.randint(0,2,len(DS_2))
    
    t='target'
    DS_2[t+'_2']=(DS_2[t]=='away').astype('int')*(-1)+(DS_2[t]=='home').astype('int')
    
    c='swap'
    
    #times -1
    Features=['Rating_diff','diff_num_partidos_tres_semanas','diff_num_partidos_diez_dias','diff_num_partidos_cuatro_dias','target_2']
    for f in Features:
        DS_2[f]=DS_2[f]*(DS_2[c]==0).astype('int')+DS_2[f]*(DS_2[c]==1).astype('int')*(-1)
    
    #1 to 0 and 0 to 1
    Features=['EqA_Local']
    for f in Features:
        DS_2[f]=DS_2[f]*(DS_2[c]==0).astype('int')+np.abs(DS_2[f]-1)*(DS_2[c]==1).astype('int')
    
    #intercambio away/home
    Features=['coach_continuity','partidos_tres_semanas','partidos_diez_dias','partidos_cuatro_dias']
    for f in Features:
        h_cc='home_'+f
        a_cc='away_'+f
        aux_h='aux'
    
        DS_2[aux_h]=DS_2[h_cc]
            
        DS_2[h_cc]=DS_2[h_cc]*(DS_2[c]==0).astype('int')+(DS_2[c]==1).astype('int')*DS_2[a_cc]

        DS_2[a_cc]=DS_2[a_cc]*(DS_2[c]==0).astype('int')+(DS_2[c]==1).astype('int')*DS_2[aux_h]

        DS_2=DS_2.drop(columns=[aux_h]) 
    
    #intercambio away/home
    historic_feature=['goal_diff','rating_diff','result_ponderado','coach_continuity','team_history_is_play_home','team_history_is_cup','outcome_V','outcome_D','relevance']
    h='home'
    a='away'
    for f in historic_feature:
        for i in range(1,11):
            l='_'+f+'_{}'.format(i)
            aux_h=h+l+'_aux'
            
            DS_2[aux_h]=DS_2[h+l]
            
            DS_2[h+l]=DS_2[h+l]*(DS_2[c]==0).astype('int')+(DS_2[c]==1).astype('int')*DS_2[a+l]
            
            DS_2[a+l]=DS_2[a+l]*(DS_2[c]==0).astype('int')+(DS_2[c]==1).astype('int')*DS_2[aux_h]
            
            DS_2=DS_2.drop(columns=[aux_h]) 
            
    #mapear targets
    t='target'
    map_dic={-1:'EqA_Derrota',0:'EqA_Empate',1:'EqA_Victoria'}
    DS_2[t]=DS_2[t+'_2'].map(map_dic)
    
    #cambiar nombres a eq1 y eq2
    Names={}
    for c in DS_2.columns:
        c2=c.replace('home','Equipo_A')
        c3=c2.replace('away','Equipo_B')
        if(  'team_history_is_play' in c3  ):
            c3=c3.replace('team_history_is_play_Equipo_A','play_home')
        elif(  'team_history_is_cup' in c3  ):
            c3=c3.replace('team_history_is_cup','is_cup')
        Names[c]=c3
    DS_2=DS_2.rename(columns=Names)
    
    #Shufle and Reset index
    DS_2 = DS_2.sample(frac=1).reset_index(drop=True)

    #Drop columnas
    DS_2=DS_2.drop(columns=['swap','target_2']) 
    
    #print
    display(DS_2)
    
    return DS_2

In [27]:
def Balance_de_empates_por_under_and_over_sampling(DS):
    '''
    Balancea las clases "Empate" con Derrota y Victoria a través de realizar un oversampling de los empates y un 
    undersampling de las derrotas y victorias. Todas las clases son afectadas porcentualmente de la misma manera. 
    
    Devuelve el Dataframe resultante.
    
    Arguments:
        DS: DataFrame
        
    Returns:
        DS_2: Dataframe
    '''
    DS_2=DS.copy(deep=True)
    # oversampling de Empate
    v,q=np.unique(DS_2.target,return_counts=True)
    ddict=dict(zip(v,q))
    ratio=((ddict['EqA_Derrota']+ddict['EqA_Victoria'])/2)/ddict['EqA_Empate']
    x=(ratio-1)/(ratio+1)#proporcion a aumentar(reducir) la clase empates (victoria/derrota) para alcanzar un balance
    print('Ratio Victorias(Derrotas)/Empates= ',round(ratio,2))
    print('Proporcion a cambiar clases= ',round(100*x,2),'%')
    
    #Random selection
    DS_2['resamp_idx']=(np.random.rand(len(DS_2))>(1-x)).astype('int')
 
    #Oversampling
    DS_2=DS_2.loc[DS_2.index.repeat(1 + DS_2.resamp_idx*(DS_2.target=='EqA_Empate').astype('int'))]
    
    #Undersampling
    DS_2 = DS_2[(DS_2.target=='EqA_Empate') | (DS_2.resamp_idx==0)]
    
    #droping of auciliar feature
    DS_2=DS_2.drop(columns=['resamp_idx'])
    
    return DS_2

### Separación del *Dataset* en *train* y *test*

In [28]:
from sklearn.model_selection import StratifiedShuffleSplit

In [29]:
def dist_of_categories(y_train,y_test):
    '''
    Genera e imprime por pantalla un report del output para el trainset y el testset, mostrando las ocurrencias y
    distribución de clases
    
    Arguments:
        y_train: target train
        y_test: target test
    '''
    Tabla={}
    Tabla['train']=y_train.sum()
    Tabla['train %']=y_train.sum()/len(y_train)
    Tabla['test']=y_test.sum()
    Tabla['test %']=y_test.sum()/len(y_test)
    Tabla=pd.DataFrame(Tabla)
    display(Tabla)
    return

In [30]:
def Training_Test_separation(DF,print_report=True,test_size=0.2):
    '''
    Separa el dataframe en test y train
    
    Arguments:
        DS: DataFrame
        print_report: si es "True" imprime un reporte de ambos datasets
        size: tamaño del testset
        
    Returns:
        TRAIN: train dataset
        TEST: test dataset
    '''
    X=DF.drop(columns=['target'])
    y=DF.target
    
    sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=42)
    sss.get_n_splits(X, y)
    
    for train_index, test_index in sss.split(X, y):
        print("TRAIN:", len(train_index), "TEST:", len(test_index))
        TRAIN=DF.iloc[train_index]
        TEST=DF.iloc[test_index]
        if(print_report):
            dist_of_categories(pd.get_dummies(y[train_index]),pd.get_dummies(y[test_index]))
    return TRAIN,TEST

In [31]:
def Split_input_and_OHE_output(DS_train,DS_test,print_report=True):
    '''
    Separa los train y test datasets en input y output, realizando un one-hot encoding en el segundo
    
    Arguments:
        DS_train: Dataset train
        DS_test: Dataset test
        print_report: si es "True" imprime un reporte sobre los outputs
        
    Returns:
        X_train: train dataset input
        y_train: target train
        X_test: test dataset input
        y_test: target test
    '''
    
    X_train=DS_train.drop(columns=['target'])
    y_train=DS_train.target
    
    X_test=DS_test.drop(columns=['target'])
    y_test=DS_test.target
    
    y_train=pd.get_dummies(y_train)
    y_test=pd.get_dummies(y_test)
    
    if(print_report):
        dist_of_categories(y_train,y_test)
        
    return X_train,y_train,X_test,y_test

### Transformación y estandarización para cada modelo

In [36]:
def Prepara_DF_para_Random_Forest(DF_train,DF_test,Features_His,Features_Match,Features_His_to_std,Features_Match_to_std):
    '''
    Prepara y estandariza un Dataframe  para modelarlo por Random Forest
    
    Arguments:
        DF: DataFrame train
        DF: DataFrame test
        Features_His: Features partidos anteriores
        Features_Match: Features partido a modelar
        Features_His_to_std: Features historicas a estandarizar
        Features_Match_to_std: Features del partido a estandarizar:
        
    Returns:
        X : train input 
        y:: train input 
        X : test input 
        y:: test input 
    '''
    y=DF_train.target
    X=DF_train.drop(columns=['target','id'])
    
    y_t=DF_test.target
    X_t=DF_test.drop(columns=['target','id'])
    
    C=[]
    for f in Features_His:
        for i in range(1,11):
            if((i!=10 )|(f!='coach_continuity')):
                C=C+[t+'_'+f+'_'+str(i) for t in ['Equipo_A','Equipo_B']]
    C=C+Features_Match
    
    X=X[C]
    X_t=X_t[C]
    
    Std_FMP=Standarizador_normal()
    
    Cs=[]
    for f in Features_His_to_std:
            Cs=Cs+[t+'_'+f+'_'+str(i) for t in ['Equipo_A','Equipo_B'] for i in range(1,11)]
    Cs=Cs+Features_Match_to_std
    for c in C:
        Std_FMP.fit(X,c)
        Std_FMP.transform(X,c)
        Std_FMP.transform(X_t,c)
    
    print('train')
    check_NaNs(X)
    print('test')
    check_NaNs(X_t)
    return X,y,X_t,y_t

In [37]:
def generate_input_for_NN(X_train,X_test,H_Features,N_Features,oldest_match=10,standarize=True):
    '''
    Da forma a un dataframe para servir de input de una red neuronal densa. Permite estandarizar las variables. 
    Para las variables históricas, la standarización se hace sobre las 10 columnas de los 10 partidos todo al mismo tiempo.
    
    Arguments:
        X_train: train input
        X_test: test input
        H_Features: Features historicas a considerar
        N_Features: Features del partido a considerar
        oldest_match: Numero de partidos historicos a considerar
        standarize: estandarizacion

        
    Returns:
        X_train_out : train input 
        X_test_out: test input 
    '''
    C=[]
    for f in N_Features:
        C=C+[f]
    for f in H_Features:
        C=C+[t+'_'+f+'_'+str(i) for t in ['Equipo_A','Equipo_B'] for i in range(1,oldest_match+1)]
    X_train_out=X_train[C]
    X_test_out=X_test[C]
    if (standarize==True):
        S=Standarizador()
        for f in H_Features:
            C=[t+'_'+f+'_'+str(i) for t in ['Equipo_A','Equipo_B'] for i in range(1,oldest_match+1)]
            S.fit(X_train_out[C],f)
            X_train_out[C]=S.transform(X_train_out[C],f)
            X_test_out[C]=S.transform(X_test_out[C],f)
        for f in N_Features:
            S.fit(X_train_out[f],f)
            X_train_out[f]=S.transform(X_train_out[f],f)
            X_test_out[f]=S.transform(X_test_out[f],f)
    
    
    
    X_train_out=X_train_out.to_numpy()
    X_test_out=X_test_out.to_numpy()
    
    print('Train shape:', np.shape(X_train_out))
    print('Test shape:', np.shape(X_test_out))
    
    return X_train_out,X_test_out

In [38]:
def generate_input_for_RNN(X_train,X_test,Features,Equipos,oldest_match=10,standarize=True):
    '''
    Da forma a un dataframe para servir de input de una red neuronal recurrente. Permite estandarizar las variables. 
    La standarización se hace sobre todas la columnas de los partidos todo al mismo tiempo.
    
    Arguments:
        X_train: train input
        X_test: test input
        Features: Features historicas a considerar
        Equipos: Equipos a considerar (A, B, o AyB), lista
        oldest_match: Numero de partidos historicos a considerar
        standarize: estandarizacion

        
    Returns:
        M : train input 
        M_t: test input 
    '''
    X={}
    X_t={}
    for f in Features:
        C=[t+'_'+f+'_'+str(i) for t in Equipos for i in range(1,oldest_match+1)]
        X[f]=X_train[C]
        X_t[f]=X_test[C]

    if (standarize==True):
        S=Standarizador()
        for f in Features:
            S.fit(X[f],f)
            X[f]=S.transform(X[f],f)
            X_t[f]=S.transform(X_t[f],f)
    M=[]
    M_t=[]
    for i in range(oldest_match,0,-1):
        L=[]
        L_t=[]
        for t in Equipos:
            for f in Features:
                c=t+'_'+f+'_'+str(i)
                L.append(X[f][c])
                L_t.append(X_t[f][c])
        M.append(L)
        M_t.append(L_t)
        
    M=np.array(M)
    M_t=np.array(M_t)
    
    M=M.transpose((2,0,1))
    M_t=M_t.transpose((2,0,1))
    
    print('Train shape:', np.shape(M))
    print('Test shape:', np.shape(M_t))
    
    return M, M_t

### Principal component analysis

In [35]:
def PCA_analisis(X,X_t,Action='Fit',umbral=100):
    '''
    Hace un Principal component analysis (PCA) del dataframe. Y reduce la dimensionalidad dependiendo el umbral dado
    
    Arguments:
        X: DF input
        Action: Fit o Fit_Transform
        umbral: umbral para los autovalores conservados al hacer transform
        
    Returns:
        X_train_out : train input 
        X_test_out: test input 
    '''
    if(Action=='Fit'):
        nf = X.shape[1]
        pca = PCA(n_components=nf)
        pca.fit(X)
        print(pca.singular_values_)
    if(Action=='Fit_Transform'):
        nf = X.shape[1]
        pca = PCA(n_components=nf)
        pca.fit(X)
        n=np.sum(pca.singular_values_>umbral)
        pca = PCA(n_components=n)
        pca.fit(X)
        X_new=pca.transform(X)
        X_t_new=pca.transform(X_t)
        return X_new,X_t_new

### Others

In [44]:
class Standarizador_normal:
    '''
    Standarizador estandar.
    '''
    def __init__(self):
        self.std =  {}
        self.mean = {}
    
    def fit(self,X,name):
        '''
        Calcula media y desviación standar de una feature
        
        Arguments:
            X: DataFrame input
            name: feature name
        '''
        self.mean[name]=np.mean(X[name])
        self.std[name]=np.std(X[name])

    def transform(self,X,name):
        '''
        transforma una feature
        
        Arguments:
            X: DataFrame input
            name: feature name
            
        Return:
            transformed output
        '''
        X[name]=(X[name]- self.mean[name])/self.std[name]
        return 

In [45]:
class Standarizador:
    '''
    Standarizador especial para operar sobre todas las columnas históricas de un tipo, al mismo tiempo.
    '''
    def __init__(self):
        self.std =  {}
        self.mean = {}
    
    def fit(self,X,name):
        '''
        Calcula media y desviación standar de una feature
        
        Arguments:
            X: input
            name: feature name
        '''
        self.mean[name]=np.mean(X.values)
        self.std[name]=np.std(X.values)

    def transform(self,X,name):
        '''
        transforma una feature
        
        Arguments:
            X: input
            name: feature name
            
        Return:
            transformed output
            
        '''
        return (X- self.mean[name])/self.std[name]
        

## MODELOS

In [32]:
#!pip install tensorflow

In [33]:
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, SimpleRNN, Dropout ,concatenate,LSTM, Reshape

from tensorflow.keras import metrics, Input, Model

from tensorflow.keras import losses, optimizers 

from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.backend import clear_session

from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier

from sklearn.decomposition import PCA

In [34]:
def clean_all_models():
    '''
    Borra todos los modelos de la memoria
    '''
    clear_session()
    return

### Random Forest

In [39]:
def Cross_Validation_RandomForest(Grid,X,y,k=7):
    
    '''
    Hace un cross validation para un Random Forest
    
    Arguments:
        Grid= grid de parametros para probar
        X= Train input
        y= Train output
        k= orden del K-fold cross validation (numeor de splits)
        
    Returns:
        Hiper-parametros optimos
    '''
    forest_clas = RandomForestClassifier(random_state=42,n_jobs=4)
    
    grid_search = GridSearchCV(forest_clas, Grid, cv=k,scoring='accuracy',return_train_score=True,verbose=4)
    
    grid_search.fit(X, y)
    
    print("Scorer")
    print(grid_search.scorer_)
    print("Best Score")
    print(grid_search.best_score_)
    print("the best trained model:")
    print(grid_search.best_estimator_)
    
    return grid_search.best_estimator_

### Red Neuronal Densa

In [40]:
def create_NN(output_units,hidden_units,activation,drop_out,input_shape,met):
    '''
    Crea una red neuronal densa
    
    Arguments:
        output_units: output shape
        hidden_units: numero de nodos en cada layer
        activation: lista con las funciones de activacion de las layers
        drop_out: lista con los dropout de las layers
        input_shape: tamaño del input (numero de features o columnas)
        met: metricas

        
    Returns:
        model: modelo 
    '''
    model = Sequential()
    N=np.arange(len(hidden_units))
    for h,a,d,i in zip(hidden_units,activation,drop_out,N):
        if(i==0):
            model.add(Dense(h, activation=a, input_shape=(input_shape,)))
        else:
            model.add(Dense(h, activation=a))
        if(d>0):
            model.add(Dropout(d))
    model.add(Dense(output_units, activation='softmax'))
    
    model.summary()
    
    adam = optimizers.Adam(learning_rate=2*10**-4)
    model.compile(loss='categorical_crossentropy', optimizer= adam, metrics=met)
    
    return model

### Modelo Red Recurrente Simple

In [41]:
def create_RNN(hidden_units_RNN,hidden_units_dense_extra, activation_dense_extra,drop_out, output_units, input_shape,met):
    '''
    Crea una red neuronal recurrente simple
    
    Arguments:
        hidden_units_RNN: layers recurrentes
        hidden_units_dense_extra: layers densas
        activation_dense_extra: lista con las funciones de activación de las layers densas
        drop_out: lista con los dropout de las layers densas
        output_units: dimensiones del output
        input_shape: (time_steps,predictors)
        met: metricas
        
    Returns:
        model: modelo 
    '''
    model = Sequential()
    model.add(SimpleRNN(hidden_units_RNN, input_shape=input_shape, dropout=0.1, recurrent_dropout=0.1)) #,activation=activation[0]
    
    for h,a,d in zip(hidden_units_dense_extra,activation_dense_extra,drop_out):
        model.add(Dense(h, activation=a))
        if(d>0):
            model.add(Dropout(d))
            
    model.add(Dense(units=output_units, activation="softmax"))
    
    model.summary()
    
    adam = optimizers.Adam(learning_rate=2*10**-4)
    model.compile(loss='categorical_crossentropy', optimizer=adam,metrics=met)
    return model

### Modelo Red Recurrente Long short-term memory (LSTM)

In [42]:
def create_LSTM(hidden_units_RNN,hidden_units_dense_extra, activation_dense_extra,drop_out, output_units, input_shape,met):
    '''
    Crea una red neuronal recurrente Long short-term memory
    
    Arguments:
        hidden_units_RNN: layers recurrentes
        hidden_units_dense_extra: layers densas
        activation_dense_extra: lista con las funciones de activación de las layers densas
        drop_out: lista con los dropout de las layers densas
        output_units: dimensiones del output
        input_shape: (time_steps,predictors)
        met: metricas
        
    Returns:
        model: modelo 
    '''
    model = Sequential()
    model.add(LSTM(hidden_units_RNN, input_shape=input_shape, dropout=0.1, recurrent_dropout=0.1)) #,activation=activation[0]
    
    for h,a,d in zip(hidden_units_dense_extra,activation_dense_extra,drop_out):
        model.add(Dense(h, activation=a))
        if(d>0):
            model.add(Dropout(d))
            
    model.add(Dense(units=output_units, activation="softmax"))
    
    model.summary()
    
    adam = optimizers.Adam(learning_rate=2*10**-4)
    model.compile(loss='categorical_crossentropy', optimizer=adam,metrics=met)
    return model

### Modelo Mixto (RNN + Denso)

In [43]:
def Create_Mix_Model(tipo_RNN ,recurrent_units, dense_units,dense_drop_out, out_shape, input_shape_history,input_shape_now,met):
    '''
    Crea una red mixta que consta de dos capas recurrentes para tratar los datos temporales de cada equipo y una capa densa donde 
    se ingresan los outputs de esas capas recurrentes + se agregan datos referentes al partido a modelar.
    
    Arguments:
        tipo_RNN: Tipo de red recurrente LSTM o Simple
        recurrent_units: layers de la capa recurrente
        dense_units: lista de numero de nodos de la capa densa
        dense_drop_out: porcentage de drop out para regularizar
        out_shape: shape del output (target)
        input_shape_history: shape de los inputs de las capas recurrentes (time_steps,predictors)
        input_shape_now: shape de los inputs de la capa densa
        met: metricas a evaluar
        
    Returns:
        model: modelo 
    '''
    
    # define two sets of inputs
    a = Input(shape=input_shape_history)
    b = Input(shape=input_shape_history)
    c = Input(shape=(input_shape_now,))
    
    # the first branch operates on the first input
    if(tipo_RNN=='Simple'):
        m1 = SimpleRNN(recurrent_units, input_shape=input_shape_history, dropout=0.1, recurrent_dropout=0,use_bias=False,unroll=False,return_sequences=True,return_state=False)(a)
        m2 = SimpleRNN(recurrent_units, input_shape=input_shape_history, dropout=0.1, recurrent_dropout=0,use_bias=False,unroll=False,return_sequences=True,return_state=False)(b)
    elif(tipo_RNN=='LSTM'):
        m1 = LSTM(recurrent_units, input_shape=input_shape_history, dropout=0.1, recurrent_dropout=0,use_bias=False,unroll=True,return_sequences=True,return_state=False)(a)
        m2 = LSTM(recurrent_units, input_shape=input_shape_history, dropout=0.1, recurrent_dropout=0,use_bias=False,unroll=True,return_sequences=True,return_state=False)(b)


    M1=Reshape((input_shape_history[0]*recurrent_units,), input_shape=input_shape_history)(m1)
    M2=Reshape((input_shape_history[0]*recurrent_units,), input_shape=input_shape_history)(m2)
    
    # combine the output of the three branches
    M = concatenate([M1,M2,c])
    
    z = Dense(units=dense_units[0], activation='relu')(M)
    z = Dropout(dense_drop_out)(z)
    z = Dense(units=dense_units[1], activation='relu')(z)
    z = Dropout(dense_drop_out)(z)
    z = Dense(units=dense_units[2], activation='relu')(z)
    z = Dropout(dense_drop_out)(z)
    z = Dense(units=dense_units[3], activation='relu')(z)
    z = Dropout(dense_drop_out)(z)
    z = Dense(units=dense_units[4], activation='relu')(z)
    z = Dense(units=out_shape, activation="softmax")(z)

    
    # our model will accept the inputs of the two branches and
    # then output a single value
    model = Model(inputs=[a, b, c], outputs=z,name='Modelo_IFA')
    
    model.summary()
    
    adam = optimizers.Adam(learning_rate=1*10**-4)
    
    model.compile(loss='categorical_crossentropy', optimizer=adam,metrics=met)
    
    
    return model

## ANALISIS DE LOS RESULTADOS DE LOS MODELOS

In [46]:
from sklearn import metrics

### Métricas

In [48]:
def compare_to_vect(y_l,y_p):
    '''
    Compara target con predicción
    Devuelve un diccionario con el "accuracy" y otra metrica más
    
    Arguments:
        y_1: target
        y_p: output de probabilidades

    Return:
        R: diccionario con resultados

    '''
    R={}
    y2=from_prob_to_outcome(y_p)
    y11=np.array(y_l)
    y22=np.array(y2)
    y33=np.array(y_p)
    s=0
    l=len(y11)
    for i in range(l):
        s=s+np.sum(np.abs(y11[i]-y22[i]))/2
    R['Acc']=(l-s,(l-s)/l)
    s=0
    for i in range(l):
        s=s+np.sum(np.abs(y11[i]-y33[i]))
        #s=s+np.sqrt(np.sum((y11[i]-y33[i])**2))
    R['Abs_mean_distance']=s/l
    return R

In [49]:
def Accuracy(y,y_p,y_t,y_t_p):
    '''
    Calcula accuracy y otra metrica para el train y test, y lo imprime por pantalla
    
    Arguments:
        y: target train
        y_p: predicciones train
        y_t: target test
        y_t_p: predicciones test

    '''
    print('ACCURACY\n')
    print('Train:')
    print(compare_to_vect(y,y_p))
    print('Test:')
    print(compare_to_vect(y_t,y_t_p))
    return

In [51]:
def draw_confusion(y,y_p,y_t,y_t_p):
    '''
    Imprime matrix de confusion
    
    Arguments:
        y: target train
        y_p: predicciones train
        y_t: target test
        y_t_p: predicciones test

    '''
    labels=labels_confusion_matrix(y)    
    fig, axs = plt.subplots(nrows = 1,ncols = 2)
    for title,yy,yy_p,a in zip(['Train','Test'],[y,y_t],[y_p,y_t_p],axs):
    
        y_hat = from_prob_to_outcome(yy_p)
        y_hat_class = np.argmax(y_hat, axis = 1)
        y_class = np.argmax(np.array(yy), axis = 1)
        
        cm = metrics.confusion_matrix(y_class, y_hat_class)
         
        a.matshow(cm)
        
        a.set_title('Confusion Matrix '+title,size=15)
        a.set_xticklabels([''] + labels, size=15)
        a.set_yticklabels([''] + labels, size=15)
        a.set_ylabel('Predicted',size=15)
        a.set_xlabel('True',size=15)
        for i in range(3):
            for j in range(3):
                a.text(i, j, cm[i,j], va='center', ha='center',color='white',size=20)
    fig.set_size_inches(10,10)
    plt.subplots_adjust(wspace=0.5)
    plt.show()

In [53]:
def Plot_metric_Model(history):
    '''
    Plotea loss y accuracy en funcion del epoch
    
    Arguments:
        history: return del fit de modelo

    '''
    fig, ax = plt.subplots(1, 2, figsize=(20,4))
    
    metric=['loss','accuracy']    
    pos=["upper right","upper left"]
    for a,m,p in zip(ax,metric,pos):
        a.plot(history.history[m], '-o',label='Training data')
        a.plot(history.history['val_'+m],'-o', label='Validation data')
        a.set_title('Training Evolution - '+m,fontsize=16)
        a.set_ylabel('Value',fontsize=14)
        a.set_xlabel('No. epoch',fontsize=14)
        a.legend(loc=p)
    plt.show()
    return

### Tablas de Reporte

In [57]:
def Report_RF_results(model,model_name,X,y,X_t,y_t,Rep):
    '''
    Realiza un reporte del RF
    
    Arguments:
        model: RF trained model
        model_name: nombre especifico de ese modelo
        X: train input
        y: train target
        X_t: test input
        y_t: test target
        Rep: Dict donde llenar el reporte
    
    Returns:
        Rep: Dict de reporte
    '''
    
    y_p=model.predict(X)
    
    
    
    Rep['Nombre'].append(model_name+' Train')
    Rep['Accuracy'].append(model.score(X,y))
    Rep['F1_macro'].append(metrics.f1_score(y,y_p,average='macro'))
    #Rep['AUC'].append(metrics.roc_auc_score(y,y_p))
    
    y_p=model.predict(X_t)
    
    
    Rep['Nombre'].append(model_name+' Test')
    Rep['Accuracy'].append(model.score(X_t,y_t))
    Rep['F1_macro'].append(metrics.f1_score(y_t,y_p,average='macro'))
    #Rep['AUC'].append(metrics.roc_auc_score(y,y_p))
    
    display(pd.DataFrame(Rep))
                               
    return Rep

In [52]:
def Print_Report(y, y_pred,y_t, y_t_pred):
    '''
    Imprime reporte de metricas para el train y test
    
    Arguments:
        y: target train
        y_pred: predicciones train
        y_t: target test
        y_t_pred: predicciones test

    '''
    ll=labels_confusion_matrix(y)
    
    l_train=[l+'_train' for l in ll]
    y_hat = from_prob_to_outcome(y_pred)
    Tr=metrics.classification_report(y,y_hat, output_dict=True,target_names=l_train)
    Tr=pd.DataFrame(Tr)

    l_test=[l+'_test' for l in ll]
    y_hat = from_prob_to_outcome(y_t_pred)
    Te=metrics.classification_report(y_t,y_hat, output_dict=True,target_names=l_test)
    Te=pd.DataFrame(Te)
    
    display(Tr)
    display(Te)
    return

In [55]:
def Append_line_to_report_table(DF,diccionario,name,RNN,partidos,N_Features_h,N_Features_m):
    '''
    Agrega una linea al reporte de resultados
    
    Arguments:
        DF: DataFrame de reporte
        diccionario: diccionario con resultados de un modelo
        name: nombre del modelo al que corresponden los resultados
        RNN: tipo de red neuronal recurrente utilizada
        partidos: número de partidos considerados
        N_Features_h: número de variables por partido del historial consideradas
        N_Features_m: número de variables referentes al partido a modelar consideradas.
    
    Returns:
        DF: Reporte con la linea nueva anexada
    '''
    diccionario['Partidos Considerados']=partidos
    diccionario['RNN']=RNN
    diccionario['N_Features_m']=N_Features_m
    diccionario['N_Features_h']=N_Features_h
    new_line=pd.DataFrame(diccionario, index=[name,])
    DF=DF.append(new_line)
    display(DF)
    return DF

In [ ]:
def Append_line_to_report_table_2(DF,diccionario,name,RNN,partidos,N_Features_h,N_Features_m,batch_size):
    '''
    Agrega una linea al reporte de resultados
    
    Arguments:
        DF: DataFrame de reporte
        diccionario: diccionario con resultados de un modelo
        name: nombre del modelo al que corresponden los resultados
        RNN: tipo de red neuronal recurrente utilizada
        partidos: número de partidos considerados
        N_Features_h: número de variables por partido del historial consideradas
        N_Features_m: número de variables referentes al partido a modelar consideradas.
        batch_size: Número de muestras que se pasan al modelo por iteración de aprendizaje
    
    Returns:
        DF: Reporte con la linea nueva anexada
    '''
    diccionario['Partidos Considerados']=partidos
    diccionario['RNN']=RNN
    diccionario['N_Features_m']=N_Features_m
    diccionario['N_Features_h']=N_Features_h
    diccionario['Batch_size']=batch_size
    new_line=pd.DataFrame(diccionario, index=[name,])
    DF=DF.append(new_line)
    display(DF)
    return DF

In [56]:
def plot_report_table(DF,columna,Y_label,RNN,N_h,N_m):
    '''
    Realiza un grafico de una columna del dataframe vs un X pasado por variable
    
    Arguments:
        DF: DataFrame de reporte
        columna: Y del plot
        Y_label: nombre de Y
        RNN: lista de tipos de RNN a graficar
        N_h: lista de numero de parámetros históricos a considerar
        N_m: lista de numero de parámetros del partido a considerar
    '''
    plt.figure(figsize=(8, 6), dpi=100)
    for r in RNN:
        for h in N_h:
            for m in N_m:  
                lab='Tipo de RNN: '+r+r',  # V x $P_h$: '+str(h)+r',  # V x $P_m$: '+str(m)
                if(r=='Simple'):
                    mark='-s'
                elif(r=='LSTM'):
                    mark='-o'
                plt.plot(DF['Partidos Considerados'][(DF['RNN']==r) & ((DF['N_Features_h']==h) & (DF['N_Features_m']==m))],DF[columna][(DF['RNN']==r) & ((DF['N_Features_h']==h) & (DF['N_Features_m']==m))],mark, label= lab)
    plt.xlabel('Numero de partidos considerados',fontsize=14)
    plt.ylabel(Y_label,fontsize=14)
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()

### Otros

In [54]:
from tensorflow.keras.callbacks import EarlyStopping

def Full_train_and_Report(model,X_train,y_train,X_test,y_test,batch_size,epochs,validation_split,num_classes=3,history_plot=True,confusion_matrix=True,Report_print=True):
    '''
    Entrena un modelo y reporta el resultado
    
    Arguments:
        model: Modelo
        X_train: train input
        y_train: train target
        X_test: test input
        y_test: test target
        batch_size: batch size
        epochs: epochs
        validation_split: tamaño set de validacion
        num_classes: numero de clases en el target
        history_plot: bool , graficar evolucion del training
        confusion_matrix: bool, graficar matrix de confusion
        Report_print: bool, imprimir reporte de test y train
    
    Returns:
        y_prediction_train: train prediction
        y_prediction_test: test prediction
        D: Diccionario con un reporte de resutados
    '''
    cb=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)]
    history = model.fit(X_train, y_train, epochs = epochs , shuffle=True , batch_size = batch_size,  verbose=1, validation_split=validation_split, callbacks=cb)
    
    if(history_plot):
        Plot_metric_Model(history)
    
    y_prediction_train=model.predict(X_train)
    y_prediction_test=model.predict(X_test)
    
    if(confusion_matrix):
        draw_confusion(y_train,y_prediction_train,y_test,y_prediction_test)
    if(Report_print):
        Print_Report(y_train,y_prediction_train,y_test,y_prediction_test)
    
    D={}
    for m in model.metrics:
        name=m.name
        D[name+' train']=m(y_train,y_prediction_train).numpy()
        D[name+' test']=m(y_test,y_prediction_test).numpy()
        
    return  y_prediction_train, y_prediction_test,D

### Auxiliar

In [50]:
def labels_confusion_matrix(y):
    '''
    Genera lista de las clases del target ordenadas
    
    Arguments:
        y: target train
    
    Returns:
        labels: lista de labels
    '''
    labels=[]
    D={'EqA_Derrota':'Eq_B_win','EqA_Empate':'Empate','EqA_Victoria':'Eq_A_win'}
    for l in list(y.columns):
        labels.append(D[l])
    return labels

In [47]:
def from_prob_to_outcome(y_p):
    '''
    transforma output de probabilidades en output de outcomes
        
    Arguments:
        y_p: output de probabilidades

    Return:
        y_outcome: output de outcomes

    '''
    y_outcome=(y_p-np.max(y_p,axis=1).reshape(len(y_p),1)==0)*1
    return y_outcome